In [34]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(Metrics)
library(rsq)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_num_finale.Rdata")

df_num <- df_num_finale

# Feature selection

In [35]:
library(leaps)

Best_Subset <-
    regsubsets(degenzaPostTraOSP~.,
               data =df_num,
               nbest = 1,      # 1 best model for each number of predictors
               nvmax = NULL,    # NULL for no limit on number of variables
               force.in = NULL, force.out = NULL,
               method = "exhaustive")
summary_best_subset <- summary(Best_Subset)

which.max(summary_best_subset$adjr2)

[1] 16

In [36]:
tmp <- summary_best_subset$which[18,]

tmp

(Intercept)                      age            degenzaPreTra 
                    TRUE                    FALSE                     TRUE 
                Sodiemia            ratioPaO2FiO2         platCountTranspl 
                   FALSE                     TRUE                     TRUE 
                albumina          surgeryDuration IntraoperativTransfected 
                    TRUE                     TRUE                    FALSE 
            plasmaFresco              plasmaPools           doseFibrinogen 
                    TRUE                     TRUE                     TRUE 
        CRIOprecipitated         CristalloidiComp           Albumina50Comp 
                    TRUE                     TRUE                     TRUE 
  noradrenalineEndSurger         intermedLactates             coldIschemia 
                    TRUE                     TRUE                     TRUE 
           organDonorAge         organDonorHeight organDonorDrawalSodiemia 
                    TRUE                     TRUE                     TRUE 
       organDonorGammaGT                 realMELD 
                   FALSE                     TRUE

In [37]:
df_num_select <- subset(df_num, select= c(degenzaPreTra,Sodiemia,ratioPaO2FiO2,platCountTranspl,surgeryDuration,IntraoperativTransfected,plasmaFresco,plasmaPools,CRIOprecipitated,CristalloidiComp,Albumina50Comp,noradrenalineEndSurger,intermedLactates,organDonorAge,organDonorHeight,organDonorDrawalSodiemia,organDonorGammaGT,realMELD,degenzaPostTraOSP))

In [38]:
dim(df_num_select)

[1] 1148   19

# Data partition

In [39]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df_num$degenzaPostTraOSP, SplitRatio = 0.7)
train  <- subset(df_num, sample == TRUE)
test   <- subset(df_num, sample == FALSE)


In [40]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample_select <- sample.split(df_num_select$degenzaPostTraOSP, SplitRatio = 0.7)
train_select  <- subset(df_num_select, sample_select == TRUE)
test_select   <- subset(df_num_select, sample_select == FALSE)


# Model

In [41]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [42]:
lm <- train(degenzaPostTraOSP ~ ., data = train,
               method = "lm",
               trControl = ctrl)

+ Fold01.Rep1: intercept=TRUE 
- Fold01.Rep1: intercept=TRUE 
+ Fold02.Rep1: intercept=TRUE 
- Fold02.Rep1: intercept=TRUE 
+ Fold03.Rep1: intercept=TRUE 
- Fold03.Rep1: intercept=TRUE 
+ Fold04.Rep1: intercept=TRUE 
- Fold04.Rep1: intercept=TRUE 
+ Fold05.Rep1: intercept=TRUE 
- Fold05.Rep1: intercept=TRUE 
+ Fold06.Rep1: intercept=TRUE 
- Fold06.Rep1: intercept=TRUE 
+ Fold07.Rep1: intercept=TRUE 
- Fold07.Rep1: intercept=TRUE 
+ Fold08.Rep1: intercept=TRUE 
- Fold08.Rep1: intercept=TRUE 
+ Fold09.Rep1: intercept=TRUE 
- Fold09.Rep1: intercept=TRUE 
+ Fold10.Rep1: intercept=TRUE 
- Fold10.Rep1: intercept=TRUE 
Aggregating results
Fitting final model on full training set


In [43]:
lm_select <- train(degenzaPostTraOSP ~ ., data = train_select,
               method = "lm",
               trControl = ctrl)

+ Fold01.Rep1: intercept=TRUE 
- Fold01.Rep1: intercept=TRUE 
+ Fold02.Rep1: intercept=TRUE 
- Fold02.Rep1: intercept=TRUE 
+ Fold03.Rep1: intercept=TRUE 
- Fold03.Rep1: intercept=TRUE 
+ Fold04.Rep1: intercept=TRUE 
- Fold04.Rep1: intercept=TRUE 
+ Fold05.Rep1: intercept=TRUE 
- Fold05.Rep1: intercept=TRUE 
+ Fold06.Rep1: intercept=TRUE 
- Fold06.Rep1: intercept=TRUE 
+ Fold07.Rep1: intercept=TRUE 
- Fold07.Rep1: intercept=TRUE 
+ Fold08.Rep1: intercept=TRUE 
- Fold08.Rep1: intercept=TRUE 
+ Fold09.Rep1: intercept=TRUE 
- Fold09.Rep1: intercept=TRUE 
+ Fold10.Rep1: intercept=TRUE 
- Fold10.Rep1: intercept=TRUE 
Aggregating results
Fitting final model on full training set


In [44]:
lm

Linear Regression 

809 samples
 22 predictor

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 728, 728, 729, 728, 728, 728, ... 
Resampling results:

  RMSE       Rsquared   MAE      
  0.6193014  0.2430372  0.4577769

Tuning parameter 'intercept' was held constant at a value of TRUE

In [45]:
lm_select

Linear Regression 

809 samples
 18 predictor

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 728, 728, 728, 728, 728, 728, ... 
Resampling results:

  RMSE       Rsquared  MAE      
  0.6152686  0.243001  0.4569514

Tuning parameter 'intercept' was held constant at a value of TRUE

In [46]:
# collect resamples
results <- resamples(list(lm=lm,  lm_select=lm_select))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: lm, lm_select 
Number of resamples: 10 

MAE 
               Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
lm        0.3793260 0.4141016 0.4642270 0.4577769 0.5026847 0.5221503    0
lm_select 0.4039166 0.4131602 0.4504914 0.4569514 0.4897385 0.5345695    0

RMSE 
               Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
lm        0.4753196 0.5491937 0.6477354 0.6193014 0.6849141 0.7055146    0
lm_select 0.4986879 0.5653263 0.6010543 0.6152686 0.6645118 0.7862929    0

Rsquared 
                Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
lm        0.08241238 0.1681984 0.2561815 0.2430372 0.3112666 0.3960851    0
lm_select 0.06108542 0.1946377 0.2458700 0.2430010 0.2729346 0.4504459    0


In [47]:
vimp <- varImp(lm_select)
vimp
plot(vimp, cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.4)))

lm variable importance

                         Overall
realMELD                 100.000
surgeryDuration           67.256
intermedLactates          62.242
noradrenalineEndSurger    59.069
degenzaPreTra             57.639
organDonorDrawalSodiemia  57.293
ratioPaO2FiO2             41.907
plasmaPools               34.246
Albumina50Comp            30.374
organDonorAge             28.387
organDonorHeight          15.702
plasmaFresco              13.628
CristalloidiComp          12.276
platCountTranspl           7.618
Sodiemia                   4.605
organDonorGammaGT          4.554
IntraoperativTransfected   1.605
CRIOprecipitated           0.000

In [48]:
test_select$predicted <- predict(lm_select, newdata = test_select)
cor(test_select$degenzaPostTraOSP ,test_select$predicted)^2  #rsquared

[1] 0.224765

In [49]:
mae(test_select$degenzaPostTraOSP ,test_select$predicted)

[1] 0.4447623

In [50]:
rmse(test_select$degenzaPostTraOSP, test_select$predicted)

[1] 0.5763392

In [51]:
#predicted vs actual

ggplot(data = test_select, aes(x = predicted, y = degenzaPostTraOSP)) +
  geom_point(alpha = 0.2, color = "blue") +
  geom_smooth(aes(x = predicted,
                  y = degenzaPostTraOSP), color="blue") +
  geom_line(aes(x = degenzaPostTraOSP,
                y = degenzaPostTraOSP), color = "black", 
            linetype = 2, size = 1)

`geom_smooth()` using method = 'loess' and formula 'y ~ x'



In [52]:
# residuals

ggplot(data = test_select, aes(x = predicted,
                      y = predicted - degenzaPostTraOSP)) +
  geom_point(alpha = 0.2, color = "blue") +
  geom_smooth(aes(x = predicted,
                  y = predicted - degenzaPostTraOSP),
              color="black")

`geom_smooth()` using method = 'loess' and formula 'y ~ x'

